<a href="https://colab.research.google.com/github/aljawhara-ibrahim/T5_task10/blob/main/RAG__Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.76MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [4]:
!pip install langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install langchain_chroma

In [6]:
!pip install Chroma

  Preparing metadata (setup.py) ... done
  Created wheel for Chroma: filename=Chroma-0.2.0-py3-none-any.whl size=7095 sha256=b31634684ccae7d7787e25dbcb2f3a01d9f3ae978492f2fdb1e41bb3bcbcd33a
  Stored in directory: /root/.cache/pip/wheels/58/74/75/a6ab7999ae473ecbe819bc5cae9ccb902429dd6c60795f5112
Successfully built Chroma


In [7]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.9 MB/s eta 0:00:00


In [8]:
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 8.3 MB/s eta 0:00:00


In [9]:
!unzip '/content/dataset.zip' -d './content'

Archive:  /content/dataset.zip
  inflating: ./content/Dataset.csv   


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [10]:
import pandas as pd

dataset = pd.read_csv('/content/Dataset.csv')

In [11]:
dataset.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [12]:
dataset.shape

(104, 2)

In [13]:
dataset

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [14]:
import os

directory = './data/markdown'
os.makedirs(directory, exist_ok=True)

In [15]:
for i in range(0,104):

  pen = dataset['المخالفة'].iloc[i]
  charg = dataset['الغرامة'].iloc[i]

  markdown_data = f'{pen}\n\n'
  markdown_data+= f'{charg}\n\n'

  with open(f'{directory}/{i}.md','w', encoding='utf-8') as file:
    file.write(markdown_data)

In [16]:
import markdown

markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)


## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

In [18]:
from langchain_community.embeddings.sentence_transformer import (
     SentenceTransformerEmbeddings,)

from langchain_community.vectorstores import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
db = Chroma.from_documents(documents, embedding_function, persist_directory='./chrome_db')

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
<ipython-input-18-82a06a083822>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [20]:
from langchain import PromptTemplate # import PromptTemplate from langchain

PROMPT_TEMPLATE="""
اجب باللغة العربية على سؤال المستخدم بناء على السياق المعطى لك
context: {context}
Question: {question}
Your answer:

"""
prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [21]:
from langchain_groq import ChatGroq
groq_api_key = "Your Api "
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [22]:
from langchain.chains import LLMChain
model = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

<ipython-input-22-c58ea476330a>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  model = LLMChain(llm=llm,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [23]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=9)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = model.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [24]:
response = query_rag("ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟")
response

Prompt after formatting:

اجب باللغة العربية على سؤال المستخدم بناء على السياق المعطى لك
context: <p>قيادة المركبة تحت تأثير مسكر أو مخدر، أو عقاقير محذر من القيادة تحت تأثيرها.</p>
<p>الغرامة المالية 5000 - 10000 ريال</p>

<p>تجاوز حافلات النقل المدرسي عند توقفها للتحميل أو التنزيل.</p>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<p>تسيير مركبة تحدث تلويثاً للبيئة على الطرق العامة.</p>
<p>الغرامة المالية 500 - 900 ريال</p>

<p>عدم ترك السائق مسافة كافية بين مركبته والمركبة التي أمامه، تسمح له بتفادي وقوع حادث.</p>
<p>الغرامة المالية 150 - 300 ريال</p>

<p>عدم وجود تأمين ساري للمركبة.</p>
<p>الغرامة المالية 100 - 150 ريال</p>

<p>عدم الوقوف تماماً عند إشارة قف.</p>
<p>الغرامة المالية 500 - 900 ريال</p>

<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.</p>
<p>الغرامة المالية 500 - 900 ريال</p>

<p>تركيب تجهيزات في المركبة كتلك الخاصة بالمركبات الرسمية ومركبات الطوارئ.</p>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<p>عدم تقيد المشاة بالإ

{'context': '<p>قيادة المركبة تحت تأثير مسكر أو مخدر، أو عقاقير محذر من القيادة تحت تأثيرها.</p>\n<p>الغرامة المالية 5000 - 10000 ريال</p>\n\n<p>تجاوز حافلات النقل المدرسي عند توقفها للتحميل أو التنزيل.</p>\n<p>الغرامة المالية 3000 - 6000 ريال</p>\n\n<p>تسيير مركبة تحدث تلويثاً للبيئة على الطرق العامة.</p>\n<p>الغرامة المالية 500 - 900 ريال</p>\n\n<p>عدم ترك السائق مسافة كافية بين مركبته والمركبة التي أمامه، تسمح له بتفادي وقوع حادث.</p>\n<p>الغرامة المالية 150 - 300 ريال</p>\n\n<p>عدم وجود تأمين ساري للمركبة.</p>\n<p>الغرامة المالية 100 - 150 ريال</p>\n\n<p>عدم الوقوف تماماً عند إشارة قف.</p>\n<p>الغرامة المالية 500 - 900 ريال</p>\n\n<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.</p>\n<p>الغرامة المالية 500 - 900 ريال</p>\n\n<p>تركيب تجهيزات في المركبة كتلك الخاصة بالمركبات الرسمية ومركبات الطوارئ.</p>\n<p>الغرامة المالية 3000 - 6000 ريال</p>\n\n<p>عدم تقيد المشاة بالإشارات الخاصة بهم.</p>\n<p>الغرامة المالية 100 - 150 ريال</p>'

In [25]:
print(f'Context:\n{response["context"]}\n\nQuestion:\n{response["question"]}\n\nText: \n{response["text"]}')

Context:
<p>قيادة المركبة تحت تأثير مسكر أو مخدر، أو عقاقير محذر من القيادة تحت تأثيرها.</p>
<p>الغرامة المالية 5000 - 10000 ريال</p>

<p>تجاوز حافلات النقل المدرسي عند توقفها للتحميل أو التنزيل.</p>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<p>تسيير مركبة تحدث تلويثاً للبيئة على الطرق العامة.</p>
<p>الغرامة المالية 500 - 900 ريال</p>

<p>عدم ترك السائق مسافة كافية بين مركبته والمركبة التي أمامه، تسمح له بتفادي وقوع حادث.</p>
<p>الغرامة المالية 150 - 300 ريال</p>

<p>عدم وجود تأمين ساري للمركبة.</p>
<p>الغرامة المالية 100 - 150 ريال</p>

<p>عدم الوقوف تماماً عند إشارة قف.</p>
<p>الغرامة المالية 500 - 900 ريال</p>

<p>عدم الوقوف وقوفاً تاماً عند إشارة (أمامك أفضلية) في حالة مرور مركبات على الطريق المعطاة له الأفضلية.</p>
<p>الغرامة المالية 500 - 900 ريال</p>

<p>تركيب تجهيزات في المركبة كتلك الخاصة بالمركبات الرسمية ومركبات الطوارئ.</p>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<p>عدم تقيد المشاة بالإشارات الخاصة بهم.</p>
<p>الغرامة المالية 100 - 150 ريال</p>

Question:
ماهي عقوبة عدم الو